In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('/content/drive/My Drive/Full_dataset/1.zip', 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import os
from keras import backend as K

In [ ]:
def modelo_unet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=['binary_crossentropy'], metrics=['accuracy'])
    model.summary()
    
    return model


In [ ]:
SIZE = 256

In [ ]:
image_dataset = [] 
images = os.listdir('/tmp/1/images/')
for i, image_name in enumerate(images):    
    if (image_name.split('.')[1] == 'tif'):
        print('Encontradas ' + str(len(image_dataset) + 1) + ' imagens')
        image = cv2.imread('/tmp/1/images/' + image_name, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_dataset.append(np.array(image))

image_dataset = (tf.keras.utils.normalize(np.array(image_dataset), axis=1))

Encontradas 1 imagens
Encontradas 2 imagens
Encontradas 3 imagens
Encontradas 4 imagens
Encontradas 5 imagens
Encontradas 6 imagens
Encontradas 7 imagens
Encontradas 8 imagens
Encontradas 9 imagens
Encontradas 10 imagens
Encontradas 11 imagens
Encontradas 12 imagens
Encontradas 13 imagens
Encontradas 14 imagens
Encontradas 15 imagens
Encontradas 16 imagens
Encontradas 17 imagens
Encontradas 18 imagens
Encontradas 19 imagens
Encontradas 20 imagens
Encontradas 21 imagens
Encontradas 22 imagens
Encontradas 23 imagens
Encontradas 24 imagens
Encontradas 25 imagens
Encontradas 26 imagens
Encontradas 27 imagens
Encontradas 28 imagens
Encontradas 29 imagens
Encontradas 30 imagens
Encontradas 31 imagens
Encontradas 32 imagens
Encontradas 33 imagens
Encontradas 34 imagens
Encontradas 35 imagens
Encontradas 36 imagens
Encontradas 37 imagens
Encontradas 38 imagens
Encontradas 39 imagens
Encontradas 40 imagens
Encontradas 41 imagens
Encontradas 42 imagens
Encontradas 43 imagens
Encontradas 44 image

In [ ]:
mask_dataset = []  
masks = os.listdir('/tmp/1/masks/')
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        print('Encontradas ' + str(len(mask_dataset) + 1) + ' máscaras')
        image = cv2.imread('/tmp/1/masks/'+ image_name, 0)
        mask_dataset.append(np.array(image))

mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

Encontradas 1 máscaras
Encontradas 2 máscaras
Encontradas 3 máscaras
Encontradas 4 máscaras
Encontradas 5 máscaras
Encontradas 6 máscaras
Encontradas 7 máscaras
Encontradas 8 máscaras
Encontradas 9 máscaras
Encontradas 10 máscaras
Encontradas 11 máscaras
Encontradas 12 máscaras
Encontradas 13 máscaras
Encontradas 14 máscaras
Encontradas 15 máscaras
Encontradas 16 máscaras
Encontradas 17 máscaras
Encontradas 18 máscaras
Encontradas 19 máscaras
Encontradas 20 máscaras
Encontradas 21 máscaras
Encontradas 22 máscaras
Encontradas 23 máscaras
Encontradas 24 máscaras
Encontradas 25 máscaras
Encontradas 26 máscaras
Encontradas 27 máscaras
Encontradas 28 máscaras
Encontradas 29 máscaras
Encontradas 30 máscaras
Encontradas 31 máscaras
Encontradas 32 máscaras
Encontradas 33 máscaras
Encontradas 34 máscaras
Encontradas 35 máscaras
Encontradas 36 máscaras
Encontradas 37 máscaras
Encontradas 38 máscaras
Encontradas 39 máscaras
Encontradas 40 máscaras
Encontradas 41 máscaras
Encontradas 42 máscaras
E

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 0)

IMG_HEIGHT = mask_dataset.shape[1]
IMG_WIDTH  = mask_dataset.shape[2]
IMG_CHANNELS = 3

def get_model():
    return modelo_unet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

filepath='/content/drive/My Drive/Full_dataset/historico/1/weights-improvement-{epoch:02d}-{val_loss:.3f}.hdf5'
check = tf.keras.callbacks.ModelCheckpoint(filepath, monitor= 'val_accuracy', verbose=1, save_best_only=True, mode='max')

csv = tf.keras.callbacks.CSVLogger('/content/drive/My Drive/Full_dataset/historico/1/mylogs.csv', separator=';' , append=False)

callback = [early, check, csv]

history = model.fit(X_train, y_train, 
                    batch_size = 16, 
                    callbacks = callback,
                    verbose=1, 
                    epochs=1000, 
                    validation_data=(X_test, y_test))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 16  0           ['conv2d[0][0]']                 
                                )                                                             